
# Mini‑projet — Classifieur Bayésien Naïf (Bernoulli) 🧠🧮

> **Mathématiques pour l'informatique — FSGA / Université Quisqueya**  
> **Enseignant : Geovany Batista Polo LAGUERRE — Semestre 1 — 2025–2026**

**Objectif.** Implémenter un pipeline simple de classification **Naïve Bayes Bernoulli** pour des **mots‑clés binaires** et l'utiliser pour décider d'envoyer une publicité à partir d'une requête contenant le mot *« pantalon »*.  
**Important :** le **lissage de Laplace** est **déjà implémenté** pour vous dans la classe fournie. Vous devez **l'utiliser** et en **interpréter** l'effet.



## Règles & rendu
- Travail **individuel**. Autorisés : `csv`, `math`, `collections`, `itertools`, `random`, `matplotlib` (facultatif). **Interdit :** `scikit-learn`.
- Rendez ce notebook **exécuté** (toutes les sorties présentes).  
- Nommez le fichier : `NOM_Prenom_NaiveBayes_Pantalon.ipynb`.

### Barème (rappel)
- Implémentation correcte (utilisation de la classe + pipeline) — 35 pts  
- Lissage de Laplace **utilisé** et **interprété** — 15 pts  
- Dénombrements & fréquences affichés — 15 pts  
- Démo et cas tests pertinents — 15 pts  
- Qualité du code & commentaires — 10 pts  
- Analyse & limites/pistes — 10 pts



## Données (jouet)
Nous utilisons un jeu **binaire** minimal avec deux mots‑clés : `pas_cher` et `anglais`, et la cible `achat ∈ {OUI, NON}`.

👉 La cellule suivante **écrit** le CSV sur votre environnement et l’**affiche**.


In [ ]:

import csv, os, pandas as pd
from pathlib import Path

csv_path = Path("/mnt/data/train_pantalon.csv")
rows = [
    {"id":1, "pas_cher":1, "anglais":0, "achat":"OUI"},
    {"id":2, "pas_cher":0, "anglais":1, "achat":"NON"},
    {"id":3, "pas_cher":0, "anglais":1, "achat":"NON"},
    {"id":4, "pas_cher":0, "anglais":1, "achat":"NON"},
    {"id":5, "pas_cher":1, "anglais":0, "achat":"NON"},
    {"id":6, "pas_cher":1, "anglais":1, "achat":"OUI"},
    {"id":7, "pas_cher":1, "anglais":0, "achat":"OUI"},
    {"id":8, "pas_cher":1, "anglais":0, "achat":"OUI"},
]

csv_path.parent.mkdir(parents=True, exist_ok=True)
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["id","pas_cher","anglais","achat"])
    writer.writeheader()
    writer.writerows(rows)

try:
    import pandas as pd
    df = pd.read_csv(csv_path)
    print("Fichier écrit :", csv_path)
    display(df)
except Exception as e:
    print("Fichier écrit :", csv_path, "| pandas indisponible (affichage brut)")
    with open(csv_path, "r", encoding="utf-8") as f:
        print(f.read())



## Classe fournie : `NaiveBayesBernoulli` (lissage de **Laplace** déjà implémenté)
- **À VOUS** d’**utiliser** cette classe dans le pipeline (chargement, fit, prédiction, affichage des comptes, etc.).
- Vous pouvez ajouter de **nouvelles features binaires** (facultatif).


In [ ]:

import csv
from collections import Counter, defaultdict
from math import log, exp

class NaiveBayesBernoulli:
    def __init__(self, alpha=1.0):
        self.alpha = float(alpha)
        self.classes_ = []
        self.features_ = []
        self.class_counts_ = Counter()
        self.feature_counts_ = defaultdict(lambda: Counter())
        self.n_ = 0

    def fit(self, X_list, y_list):
        self.n_ = len(y_list)
        self.classes_ = sorted(set(y_list))
        feat = set()
        for X in X_list:
            feat |= set(X.keys())
        self.features_ = sorted(feat)
        self.class_counts_.clear()
        self.feature_counts_.clear()
        for X, y in zip(X_list, y_list):
            self.class_counts_[y] += 1
            for f in self.features_:
                v = int(X.get(f, 0))
                self.feature_counts_[y][(f, v)] += 1
        return self

    def _p_class(self, c):
        return self.class_counts_[c] / self.n_

    def _p_feat_given_class(self, feat, val, c):
        c1 = self.feature_counts_[c][(feat, 1)]
        c0 = self.feature_counts_[c][(feat, 0)]
        tot = c1 + c0
        num = (c1 + self.alpha) if val == 1 else (c0 + self.alpha)
        den = tot + 2*self.alpha
        return num / den

    def predict_proba(self, X):
        scores = {}
        for c in self.classes_:
            s = log(self._p_class(c))
            for f in self.features_:
                v = int(X.get(f, 0))
                s += log(self._p_feat_given_class(f, v, c))
            scores[c] = s
        m = max(scores.values())
        exps = {c: exp(v - m) for c, v in scores.items()}
        Z = sum(exps.values())
        return {c: exps[c]/Z for c in self.classes_}

    def predict(self, X):
        proba = self.predict_proba(X)
        return max(proba, key=proba.get)

def load_csv_binary(path, feature_names=("pas_cher","anglais")):
    X_list, y_list = [], []
    with open(path, newline="", encoding="utf-8") as f:
        for row in csv.DictReader(f):
            X = {feat: int(row[feat]) for feat in feature_names}
            y = row["achat"].strip()
            X_list.append(X); y_list.append(y)
    return X_list, y_list

def pretty_counts(nb: NaiveBayesBernoulli):
    print("Classes :", nb.classes_)
    print("Features:", nb.features_)
    for c in nb.classes_:
        print(f"\nClasse {c} (count={nb.class_counts_[c]})")
        for f in nb.features_:
            c1 = nb.feature_counts_[c][(f,1)]
            c0 = nb.feature_counts_[c][(f,0)]
            tot = c1 + c0
            p1 = (c1 + nb.alpha) / (tot + 2*nb.alpha)
            p0 = (c0 + nb.alpha) / (tot + 2*nb.alpha)
            print(f"  {f}: #1={c1}, #0={c0}, p(1|{c})={p1:.3f}, p(0|{c})={p0:.3f}")



## TODO 1 — Charger les données et entraîner le modèle
1. Charger `train_pantalon.csv` avec `load_csv_binary`.  
2. Créer `NaiveBayesBernoulli(alpha=1.0)` et appeler `fit`.  
3. Afficher les **comptes** et **probabilités lissées** via `pretty_counts`.


In [ ]:

# === VOTRE CODE (TODO 1) ======================================================
X_train, y_train = load_csv_binary("/mnt/data/train_pantalon.csv", feature_names=("pas_cher","anglais"))
nb = NaiveBayesBernoulli(alpha=1.0).fit(X_train, y_train)
pretty_counts(nb)



## TODO 2 — Prédire et interpréter
1. Calculer `predict_proba` et `predict` pour les cas : [1,1], [1,0], [0,1], [0,0].  
2. **Comparer** avec/sans lissage (`alpha=0` vs `alpha=1`).  
3. En 4–6 lignes : **interpréter** l’effet du lissage.


In [ ]:

# === VOTRE CODE (TODO 2) ======================================================
tests = [
    {"pas_cher":1, "anglais":1},
    {"pas_cher":1, "anglais":0},
    {"pas_cher":0, "anglais":1},
    {"pas_cher":0, "anglais":0},
]

def run_preds(alpha):
    nb = NaiveBayesBernoulli(alpha=alpha).fit(X_train, y_train)
    out = []
    for x in tests:
        proba = nb.predict_proba(x)
        yhat = nb.predict(x)
        out.append((x, proba, yhat))
    return out

print("== Avec lissage alpha=1.0 ==")
for x, proba, yhat in run_preds(1.0):
    print(x, "→", yhat, "| proba:", {k:round(v,3) for k,v in proba.items()})

print("\n== Sans lissage alpha=0.0 ==")
for x, proba, yhat in run_preds(0.0):
    print(x, "→", yhat, "| proba:", {k:round(v,3) for k,v in proba.items()})



## TODO 3 — Ajouter une ou deux features (facultatif, recommandé)
- Proposer une nouvelle variable binaire (ex. `promo`, `dispo`, `marque`) et **étendre le CSV**.  
- Réentraîner et commenter l’impact sur les prédictions.



## TODO 4 — Rapport court (5–8 lignes)
- Hypothèse **naïve** (indépendance conditionnelle **à classe fixée**).  
- Pourquoi elle est pratique (et sa **limite**).  
- Interpréter l’effet du **lissage** à partir de vos résultats.



## (Option) Visualisation simple
Carte des scores en fonction de `pas_cher` et `anglais`.


In [ ]:

# OPTIONNEL : visualiser p(OUI | pas_cher, anglais)
import matplotlib.pyplot as plt
import numpy as np

nb = NaiveBayesBernoulli(alpha=1.0).fit(X_train, y_train)

grid = [(a,b) for a in [0,1] for b in [0,1]]
scores = []
for a,b in grid:
    proba = nb.predict_proba({"pas_cher":a,"anglais":b})
    scores.append(proba["OUI"])

fig, ax = plt.subplots(figsize=(4,3))
im = ax.imshow(np.array(scores).reshape(2,2), vmin=0, vmax=1, origin="lower")
ax.set_xticks([0,1]); ax.set_xticklabels(["anglais=0","anglais=1"])
ax.set_yticks([0,1]); ax.set_yticklabels(["pas_cher=0","pas_cher=1"])
for i,(a,b) in enumerate(grid):
    ax.text(b, a, f"{scores[i]:.2f}", ha="center", va="center", color="w")
ax.set_title("p(OUI | pas_cher, anglais) — alpha=1")
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
plt.tight_layout()
plt.show()
